# Load Video Frames into Pixeltable

In [ ]:
import sys, glob
import pandas as pd
import numpy as np
sys.path.append('/home/marcel/pixeltable')

In [ ]:
import pixeltable as pt
from pixeltable import catalog
from pixeltable.type_system import ImageType, StringType, IntType
%load_ext autoreload
%autoreload 2

In [ ]:
cl = pt.Client()
try:
    db = cl.get_db('videos')
except:
    db = cl.create_db('videos')

The frames will be stored in a table `data`. Initially it only contains the path to the video file, the frame, and the frame sequence number (within the video).

In [ ]:
db.drop_table('data', ignore_errors=True)
t = db.create_table('data', [
    catalog.Column('video_file', StringType(), nullable=False),
    catalog.Column('frame', ImageType(), nullable=False, indexed=True),
    catalog.Column('frame_idx', IntType(), nullable=False),
])

This demo uses 3 short videos

In [ ]:
video_filepaths = glob.glob(f'/home/marcel/pixeltable/**/videos/*.mp4', recursive=True)
video_filepaths

In [ ]:
df = pd.DataFrame({'video_file': video_filepaths})

We "load" the data by supplying the file paths for the video files.
- The `video_column` argument instructs Pixeltable to treat that data as video files from which to extract frames.
- Each input row, corresponding to one video, is expanded into one row per frame (subject to the requested frame rate, in this case 1 fps).
- Each frame is extract to a JPEG file that is stored in the Pixeltable home directory.
- The columns `frame` and `frame_idx` receive the frame file path and frame sequence number, respectively.

In [ ]:
t.insert_pandas(df, video_column='video_file', frame_column='frame', frame_idx_column='frame_idx', fps=1)

We ended up with 67 frames.

In [ ]:
t.count()

Let's look at the data

In [ ]:
t[t.frame.matches('street')].show(10)

In [ ]:
t[t.frame, t.frame.width, t.frame.height].show()